In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

In [ ]:
movies.head(1)

In [ ]:
movies["genres"][0:2]

In [ ]:
movies["keywords"][0]

In [ ]:
movies["overview"][0]

In [ ]:
movies["production_companies"][0]

In [ ]:
movies["tagline"][0]

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
credits.head()

In [ ]:
credits["crew"][0]

In [ ]:
credits.cast[0]

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(1)
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [ ]:
movies = movies[['id','original_title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
import ast

In [ ]:
def convert_(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies["genres"].dtypes

In [ ]:
movies["genres"][0]

In [ ]:
# movies['genres'] = movies['genres'].apply(convert_)
movies.head(1)

In [ ]:
movies["keywords"][0]

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert_)
movies.head(1)

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies["cast"][0]

In [ ]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head(1)

In [ ]:
movies["cast"][0:4]

In [ ]:
movies['cast'].apply(lambda x:x[0:3])

In [ ]:
movies["cast"][0]

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies["crew"][0:10]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies["overview"][0]

In [ ]:
movies.head(1)

In [ ]:

movies.sample(5)

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head(1)

In [ ]:
# movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new.head()

In [ ]:
# new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
new["tags"][0]

In [ ]:
new["tags"]=new["tags"].apply(lambda x:x.lower())

In [ ]:
new["tags"][0]

In [66]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [67]:
def stem(text):
    y=[ ]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [68]:
new["tags"]=new["tags"].apply(stem)

In [69]:
new.tags[0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [71]:
vector = cv.fit_transform(new['tags']).toarray()

In [72]:
vector.shape

(4806, 5000)

In [73]:
vector[0:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [74]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [75]:
len(cv.get_feature_names_out())

5000

In [76]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [77]:
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [78]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [83]:
new.head(1)

,id,original_title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


In [86]:
def recommend(movie):
    index = new[new['original_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].original_title)
        
    

This code defines a Python function called recommend(movie) that is used for recommending movies based on their similarity to a given movie. Here's a step-by-step explanation of how the code works:

index = new[new['original_title'] == movie].index[0]: This line is responsible for finding the index of the movie in the dataset new that matches the given movie title. It assumes that the dataset new is a DataFrame, likely from a library like Pandas, and that it has a column named 'original_title' that contains movie titles. It uses boolean indexing to find rows where the 'original_title' column matches the input movie, and then it retrieves the index of the first matching row (assuming there is at least one match).

distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1]): This line calculates the similarity between the selected movie and all other movies in the dataset, and then sorts them in descending order based on their similarity scores. Here's a breakdown of what's happening:

similarity is assumed to be a 2D list or array where each row represents a movie and each column represents a similarity score between the movie at the index and other movies in the dataset.
enumerate(similarity[index]) pairs each similarity score with its corresponding movie index.
sorted(..., reverse=True, key=lambda x: x[1]) sorts these pairs in reverse order (from highest similarity to lowest similarity) based on the second element of each pair (i.e., the similarity score).
for i in distances[1:6]:: This loop iterates over the top 5 movies (excluding the first one, which is the movie itself since it will have a similarity score of 1) with the highest similarity scores.

print(new.iloc[i[0]].original_title): For each movie in the top 5 similar movies, it retrieves the movie's title by using the index stored in i[0] and then prints the title to recommend the similar movies to the user.

In summary, this code takes a movie title as input, finds its index in a dataset, computes the similarity of this movie with all other movies in the dataset, and recommends the top 5 movies with the highest similarity scores to the input movie.

In [87]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [90]:
recommend('Interstellar')

Guardians of the Galaxy
Silent Running
Space Cowboys
The Martian
The Right Stuff


In [95]:
import pickle

In [102]:
# pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [100]:
new.head(1)

,id,original_title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


In [98]:
new.to_dict

<bound method DataFrame.to_dict of           id                            original_title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4804    9367                               El Mariachi   
4805   72766                                 Newlyweds   
4806  231617                 Signed, Sealed, Delivered   
4807  126186                          Shanghai Calling   
4808   25975                         My Date with Drew   

                                                   tags  
0     in the 22nd century, a parapleg marin is dispa...  
1     captain barbossa, long believ to be dead, ha c...  
2     a cryptic messag from bond’ past send him on a...  
3     follow the death of district a

In [96]:
new.head(1)

,id,original_title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


In [91]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/df/e2/e1c8e3abce0e819a4ef3c6a5af4ddbb1eadbca0cf2ebd5e58853edcd58bd/streamlit-1.28.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/17/16/b12fca347ff9d062e3c44ad9641d2ec50364570a059f3078ada3a5119d7a/altair-5.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-metadata<7,>=1.4 from https://files.pythonhosted.org/packages/cc/37/db7ba97e676af155f5fcb1a35466f446eadc9104e25b8336